<a href="https://colab.research.google.com/github/Kyllien/DRIM-Game/blob/main/MatriceTransitionGood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Import data from os
import os
os.chdir("/content/gdrive/My Drive/DRIM")

In [ ]:
#Librairie
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import transitionMatrix as tm
from transitionMatrix.estimators import simple_estimator as es
from transitionMatrix.estimators import cohort_estimator as ec
import datetime

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv('base_1.csv')

# print(np.unique(data['classe'])) #Aucun prob de classe

#rename the columns
data = data.rename(columns={"classe": "State", "dtf_per_trt": "Time", "ID_BCR_TRS": "ID"})
# Put Time in Date Forma
data["Time"]=pd.to_datetime(data["Time"])



In [ ]:
#Creation du Jeu Test
df = data.sample(10)
#df.to_csv('C:/Users/33640/source/repos/Kyllien/DRIM-Game/data_sample.csv')

#df = data.head(1000000)
df = df.sort_values(by=["ID","Time"])

In [ ]:
df=data
df=df.sort_values(by=["Time"])

<bound method NDFrame.head of                             ID       Time  State
129470    9NYLO0OR7S35SU3FA63C 2010-01-31      5
151892    9NYWOBAG6DND9CX35FS5 2010-01-31      7
141574    9NYV8PAXR5ZXJZYXI368 2010-01-31      2
147513    9NYWIJAXRGNCF33XI6WE 2010-01-31     11
131274    9NYLOLNEO8VNDBH5UQIX 2010-01-31      2
...                        ...        ...    ...
21977805  9NVVORQL5WN5JSQXI6W5 2017-12-31      6
22013341  9NY3IA76G5ZDQKNJ5QSU 2017-12-31      1
22056281  9R9WIJAABOQMS153NQII 2017-12-31      6
21975849  9NVVO7A7OCWHE63SW64J 2017-12-31      4
21834477  9KVQUAL3AI3MJF4MW1DJ 2017-12-31      3

[1000 rows x 3 columns]>

In [ ]:
df=data
df=df.sort_values(by=["Time"])
ID=np.unique(df["ID"])
df2=pd.DataFrame(ID,columns=["ID"])
compteur=0
for i in np.unique(df["Time"]):
  print(i)
  df3=df[df["Time"]==i]
  df3=df3[["ID","State"]]
  df3=df3.rename({'ID': 'ID', 'State': 'Date'+str(compteur)}, axis='columns')
  compteur=compteur+1
  df2=df2.merge(df3,on='ID',how='left')


2010-01-31T00:00:00.000000000
2010-02-28T00:00:00.000000000
2010-03-31T00:00:00.000000000
2010-04-30T00:00:00.000000000
2010-05-31T00:00:00.000000000
2010-06-30T00:00:00.000000000
2010-07-31T00:00:00.000000000
2010-08-31T00:00:00.000000000
2010-09-30T00:00:00.000000000
2010-10-31T00:00:00.000000000
2010-11-30T00:00:00.000000000
2010-12-31T00:00:00.000000000
2011-01-31T00:00:00.000000000
2011-02-28T00:00:00.000000000
2011-03-31T00:00:00.000000000
2011-04-30T00:00:00.000000000
2011-05-31T00:00:00.000000000
2011-06-30T00:00:00.000000000
2011-07-31T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-30T00:00:00.000000000
2011-10-31T00:00:00.000000000
2011-11-30T00:00:00.000000000
2011-12-31T00:00:00.000000000
2012-01-31T00:00:00.000000000
2012-02-29T00:00:00.000000000
2012-03-31T00:00:00.000000000
2012-04-30T00:00:00.000000000
2012-05-31T00:00:00.000000000
2012-06-30T00:00:00.000000000
2012-07-31T00:00:00.000000000
2012-08-31T00:00:00.000000000
2012-09-30T00:00:00.000000000
2012-10-31

In [ ]:
#Creation d'une classe pour les non rating : 12
df2=df2.fillna(12)

In [ ]:
#Creation Matrice PIT
PIT={}
for i in range(1,96):
  u=df2.columns[i]
  m=df2.columns[i+1]
  PIT[i]=pd.crosstab(df2[u],df2[m],normalize="index")



In [ ]:
#Creation matrice TTC
TTC=PIT[1]
for i in range(2,96):
  TTC=TTC+PIT[i]
TTC=TTC/len(PIT)
TTC

Date1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
Date0,,,,,,,,,,,,
1.0,0.918037,0.014044,0.011101,0.002218,0.000166,0.000362,0.000065,0.000041,0.000022,0.000002,0.000007,0.053935
2.0,0.025523,0.947654,0.001547,0.002833,0.004105,0.004805,0.001010,0.000078,0.000085,0.000004,0.000010,0.012347
3.0,0.004771,0.027495,0.946670,0.000338,0.000381,0.004274,0.004916,0.001363,0.000188,0.000100,0.000019,0.009486
4.0,0.000059,0.012926,0.030076,0.931962,0.000476,0.000693,0.010990,0.000104,0.003059,0.000169,0.000072,0.009413
5.0,0.000059,0.009737,0.008609,0.022137,0.933075,0.000658,0.012246,0.000416,0.004465,0.000226,0.000112,0.008261
6.0,0.000073,0.000923,0.014535,0.001806,0.015672,0.931821,0.001794,0.013944,0.006767,0.002724,0.000266,0.009676
7.0,0.000040,0.000939,0.002479,0.008797,0.009326,0.016917,0.906917,0.001875,0.022558,0.017797,0.001022,0.011334
8.0,0.000039,0.000383,0.005972,0.001114,0.001166,0.043384,0.036326,0.823006,0.014334,0.059834,0.003499,0.010943
9.0,0.000040,0.000673,0.001296,0.006093,0.009364,0.024286,0.089396,0.020483,0.697799,0.119445,0.018033,0.013092


In [ ]:
#Calcul des coefficient de correlation
dataDefaut = pd.read_excel("Taux.Defaut.xlsx")
dataDefaut


,Time,Taux_Defaut
0,2010-01-31,0.059661
1,2010-02-28,0.052415
2,2010-03-31,0.051614
3,2010-04-30,0.049995
4,2010-05-31,0.049792
...,...,...
115,2019-08-31,0.029779
116,2019-09-30,0.027923
117,2019-10-31,0.027948
118,2019-11-30,0.027663


In [ ]:
#TTC Cumule
TTC_C = TTC
TTC[12][1]
for i in range(1,13):
  compteur=0
  for j in reversed(range(1,13)):
    compteur=TTC[j][i]+compteur
    TTC_C[j][i]=compteur
TTC_C


NameError: ignored